In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split


In [ ]:
# Load the IMDB dataset from CSV
df = pd.read_csv('/content/drive/MyDrive/IMDB_Dataset.csv', encoding='utf-8',on_bad_lines='skip')


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/movie-review-dataset.csv', on_bad_lines='skip')

In [ ]:
print(len(test_df))

609


In [ ]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})

print(df.head(20))

                                               review  sentiment
0   One of the other reviewers has mentioned that ...          1
1   A wonderful little production. <br /><br />The...          1
2   I thought this was a wonderful way to spend ti...          1
3   Basically there's a family where a little boy ...          0
4   Petter Mattei's "Love in the Time of Money" is...          1
5   Probably my all-time favorite movie, a story o...          1
6   I sure would like to see a resurrection of a u...          1
7   This show was an amazing, fresh & innovative i...          0
8   Encouraged by the positive comments about this...          0
9   If you like original gut wrenching laughter yo...          1
10  Phil the Alien is one of those quirky films wh...          0
11  I saw this movie when I was about 12 when it c...          0
12  So im not a big fan of Boll's work but then ag...          0
13  The cast played Shakespeare.<br /><br />Shakes...          0
14  This a fantastic movi

In [ ]:
# Preprocess the text column
import re
import pandas as pd
import numpy as np
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove html tags
    text = re.sub('<[^<]+?>', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

df['review'] = df['review'].apply(preprocess_text)

In [ ]:
df1 = df[df['review'].str.split().str.len() < 100]

print(df1.head(10))


                                               review  sentiment
9   if you like original gut wrenching laughter yo...          1
10  phil the alien is one of those quirky films wh...          0
14  this a fantastic movie of three prisoners who ...          1
22  what an absolutely stunning movie if you have ...          1
36  the plot is about the death of little children...          0
46  protocol is an implausible movie whose only sa...          0
49  average and surprisingly tame fulci giallo whi...          0
53  i cannot believe i enjoyed this as much as i d...          1
62  so lets beginthe movie itself is as original a...          1
63  besides being boring the scenes were oppressiv...          0


In [ ]:
print(len(df1))

6510


In [ ]:
print("Number of reviews in df1:", df1.shape[0])
print("Number of positive and negative sentiments in df1:")
print(df1['sentiment'].value_counts())


Number of reviews in df1: 6510
Number of positive and negative sentiments in df1:
1    3512
0    2998
Name: sentiment, dtype: int64


In [ ]:
# Define tokenizer
import transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [ ]:
# Tokenize train dataset
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df1['review']), padding=True, truncation=True, max_length=100)
X_train = train_encodings['input_ids']
y_train = np.array(list(df1['sentiment']))
print(len(X_train))
print(len(y_train))



6510
6510


In [ ]:
# Tokenize test dataset
test_encodings = tokenizer(list(test_df['review']), padding=True, truncation=True, max_length=100)
X_val = test_encodings['input_ids']
y_val = np.array(list(test_df['sentiment']))

In [ ]:
# Train-test split
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define BERT model

In [ ]:
print(len(X_train))
print(len(y_train))

6510
6510


In [ ]:
# Define BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# Train the model
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train, "attention_mask": np.ones_like(X_train)},
    y_train
))
train_dataset = train_dataset.shuffle(len(X_train)).batch(32)

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_val, "attention_mask": np.ones_like(X_val)},
    y_val
))
val_dataset = val_dataset.batch(32)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,
    steps_per_epoch=len(X_train) // 32
)

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Model/Model-PreTrained')

In [ ]:
from transformers import TFBertForSequenceClassification

# Load saved model
model_path = '/content/drive/MyDrive/Model/Model-PreTrained'
model = TFBertForSequenceClassification.from_pretrained(model_path)
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
!pip install --upgrade textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/9

In [ ]:
!pip show textattack

Name: textattack
Version: 0.3.8
Summary: A library for generating text adversarial examples
Home-page: https://github.com/QData/textattack
Author: QData Lab at the University of Virginia
Author-email: jm8wx@virginia.edu
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: bert-score, click, datasets, editdistance, filelock, flair, jieba, language-tool-python, lemminflect, lru-dict, more-itertools, nltk, num2words, numpy, OpenHowNet, pandas, pinyin, pycld2, PySocks, scipy, terminaltables, torch, tqdm, transformers, word2number
Required-by: 


In [ ]:
import textattack.attack_recipes as attack_recipes

# Get a list of all the methods and attributes in attack_recipes.
methods = [method for method in dir(attack_recipes) if callable(getattr(attack_recipes, method))]

# Print the list of methods.
print(methods)



textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['A2TYoo2021', 'AttackRecipe', 'BAEGarg2019', 'BERTAttackLi2020', 'CLARE2020', 'CheckList2020', 'DeepWordBugGao2018', 'FasterGeneticAlgorithmJia2019', 'FrenchRecipe', 'GeneticAlgorithmAlzantot2018', 'HotFlipEbrahimi2017', 'IGAWang2019', 'InputReductionFeng2018', 'Kuleshov2017', 'MorpheusTan2020', 'PSOZang2020', 'PWWSRen2019', 'Pruthi2019', 'Seq2SickCheng2018BlackBox', 'SpanishRecipe', 'TextBuggerLi2018', 'TextFoolerJin2019']


In [ ]:
import textattack

print(dir(textattack))


['Attack', 'AttackArgs', 'Attacker', 'AugmenterArgs', 'CommandLineAttackArgs', 'CommandLineTrainingArgs', 'DatasetArgs', 'Metric', 'ModelArgs', 'Trainer', 'TrainingArgs', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'attack', 'attack_args', 'attack_recipes', 'attack_results', 'attacker', 'augment_args', 'augmentation', 'commands', 'constraints', 'dataset_args', 'datasets', 'goal_function_results', 'goal_functions', 'loggers', 'metrics', 'model_args', 'models', 'name', 'search_methods', 'shared', 'trainer', 'training_args', 'transformations']


In [ ]:
import torch
import transformers
import textattack
import transformers
from textattack.attack_recipes import A2TYoo2021
from textattack.attack_recipes import InputReductionFeng2018

In [ ]:
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import DeepWordBugGao2018

In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019

In [ ]:
# Create a new instance of BertForSequenceClassification
from transformers import BertForSequenceClassification, BertTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

# Load the saved model
# model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Model/Finalmodel')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from textattack.models.wrappers import ModelWrapper
import numpy as np

class MyModelWrapper(ModelWrapper):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __call__(self, text_input_list):
        input_ids = []
        token_type_ids = []
        attention_mask = []
        for text_input in text_input_list:
            input_dict = self.tokenizer.encode_plus(text_input, add_special_tokens=True, max_length=512, pad_to_max_length=True)
            input_ids.append(input_dict['input_ids'])
            token_type_ids.append(input_dict['token_type_ids'])
            attention_mask.append(input_dict['attention_mask'])
            
        input_ids = np.array(input_ids)
        token_type_ids = np.array(token_type_ids)
        attention_mask = np.array(attention_mask)
        outputs = self.model([input_ids, token_type_ids, attention_mask], training=False)
        logits = outputs[0]
        return logits.numpy()


In [ ]:
model_wrapper = MyModelWrapper(model, tokenizer)

In [ ]:
# Initialize the TextFooler attack recipe
attack = TextFoolerJin2019.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
text = "The movie kept me engaged the entire time"
results_iterable = attack.attack(text, 1)
print(results_iterable)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1 (81%) --> 0 (70%)

The movie kept me engaged the entire time

The movie obligated me engaged the entire time


In [ ]:
# test_df.columns = ['review', 'sentiment']
# test_df.to_csv('test1.csv', index=False)
texts = np.array(test_df['review'])
labels = np.array(test_df['sentiment'])

In [ ]:
print(texts[0])
print(labels[0])

This movie was great!
1


In [ ]:
text = texts[0]
label = labels[0]
results_iterable = attack.attack(text, ground_truth_output=int(label))
print(results_iterable)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1 (98%) --> 0 (89%)

This movie was great!

This movie was sizeable!


In [ ]:
print(dir(results_iterable))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'diff_color', 'goal_function_result_str', 'num_queries', 'original_result', 'original_text', 'perturbed_result', 'perturbed_text', 'str_lines']


In [ ]:
adv_text = results_iterable.perturbed_text()

print(adv_text)

Terrible caretaker ruins this one.


In [ ]:
adv_texts = []
for text, label in zip(texts, labels):
    results_iterable = attack.attack(text, ground_truth_output=int(label))
    if hasattr(results_iterable, "__iter__"):
        for result in results_iterable:
            adv_text = results_iterable.perturbed_text()
            adv_texts.append(adv_text)
            print(f"Adversarial example: {adv_text}")
    else:
        adv_text = results_iterable.perturbed_text()
        adv_texts.append(adv_text)
        print(f"Adversarial example: {adv_text}")



Adversarial example: This movie was sizeable!
Adversarial example: The acting was tragic
Adversarial example: I rained this film
Adversarial example: It was a jingles of time
Adversarial example: The plot was foretold
Adversarial example: These is a must-watch stills
Adversarial example: me would not applauds this photographs
Adversarial example: The cinematography was staggering
Adversarial example: Both dash was humble
Adversarial example: The ending was saddened
Adversarial example: This is the decent movie ever!
Adversarial example: I mourn watching this film
Adversarial example: Both idiosyncrasies were well-developed
Adversarial example: He was too slow-paced
Adversarial example: I was on the edge of my seat the whole time
Adversarial example: The special effects were startling
Adversarial example: I was pierce throughout the entire movie
Adversarial example: The soundtrack was leggy
Adversarial example: The dialogue was cutesy
Adversarial example: me would staring this imaging a

In [ ]:
import csv
csv_file_path = '/content/drive/MyDrive/Model/TextFoolerJin2019_adv_examples.csv'
# Write the adversarial examples to the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['review', 'sentiment']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for example in adv_texts:
        writer.writerow({'review': example[0], 'sentiment': example[1]})

In [ ]:
# Tokenize the adversarial texts
adv_inputs = tokenizer(list(adv_texts), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Evaluate accuracy using the adversarial examples
adv_preds = model_wrapper.model(adv_inputs).logits.numpy().argmax(axis=1)
adv_acc = (adv_preds == labels).mean()

print(f"Adversarial accuracy: {adv_acc:.4f}")

Adversarial accuracy: 0.5123


In [ ]:
# Create DataFrame with adv_texts and labels
df_adv = pd.DataFrame({'review': adv_texts, 'sentiment': labels})

In [ ]:
df_adv['review'] = df_adv['review'].apply(preprocess_text)
adv_df1 = df_adv[df_adv['review'].str.split().str.len() < 100]


In [ ]:
# Create a new dataframe containing both original and adversarial examples
df_adv = pd.concat([df1, adv_df1], ignore_index=True)

In [ ]:
print(df_adv.head(10))

                                              review  sentiment
0  if you like original gut wrenching laughter yo...          1
1  phil the alien is one of those quirky films wh...          0
2  this a fantastic movie of three prisoners who ...          1
3  what an absolutely stunning movie if you have ...          1
4  the plot is about the death of little children...          0
5  protocol is an implausible movie whose only sa...          0
6  average and surprisingly tame fulci giallo whi...          0
7  i cannot believe i enjoyed this as much as i d...          1
8  so lets beginthe movie itself is as original a...          1
9  besides being boring the scenes were oppressiv...          0


In [ ]:
df_adv.to_csv('/content/drive/MyDrive/Model/adv_examples.csv', index=False)

In [ ]:
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df_adv['review']), padding=True, truncation=True, max_length=100)
X_train1 = train_encodings['input_ids']
y_train1 = np.array(list(df_adv['sentiment']))

In [ ]:
# Train the model
train_dataset1 = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train1, "attention_mask": np.ones_like(X_train1)},
    y_train1
))
train_dataset1 = train_dataset1.shuffle(len(X_train1)).batch(32)

In [ ]:
history = model.fit(
    train_dataset1,
    validation_data=val_dataset,
    epochs=2,
    steps_per_epoch=len(X_train1) // 32
)

Epoch 1/2
222/222 [==============================] - 218s 735ms/step - loss: 0.3422 - accuracy: 0.8467 - val_loss: 0.1773 - val_accuracy: 0.9704
Epoch 2/2
  1/222 [..............................] - ETA: 2:30 - loss: 0.2984 - accuracy: 0.9333

222/222 [==============================] - 2s 6ms/step - loss: 0.2984 - accuracy: 0.9333 - val_loss: 0.1738 - val_accuracy: 0.9737


In [ ]:
model.save_pretrained('/content/drive/MyDrive/Model/Model-Trained-TextFoolerJin2019')

In [ ]:
# Define the model wrapper
class ModelWrapper(textattack.models.wrappers.ModelWrapper):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, text_input_list):
        input_ids = []
        attention_mask = []
        for text_input in text_input_list:
            inputs = self.tokenizer.encode_plus(text_input, add_special_tokens=True, max_length=128, padding='max_length', truncation=True)
            input_ids.append(inputs['input_ids'])
            attention_mask.append(inputs['attention_mask'])
        inputs = {
            'input_ids': tf.constant(input_ids),
            'attention_mask': tf.constant(attention_mask)
        }
        outputs = self.model(inputs)
        logits = outputs.logits.numpy()
        return logits

In [ ]:
# Initialize the DeepWordBugGao2018 attack
model_wrapper = ModelWrapper(model, tokenizer)

In [ ]:
attack = DeepWordBugGao2018.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
adv_texts = []
for text, label in zip(texts, labels):
    results_iterable = attack.attack(text, ground_truth_output=int(label))
    if hasattr(results_iterable, "__iter__"):
        for result in results_iterable:
            adv_text = results_iterable.perturbed_text()
            adv_texts.append(adv_text)
            print(f"Adversarial example: {adv_text}")
    else:
        adv_text = results_iterable.perturbed_text()
        adv_texts.append(adv_text)
        print(f"Adversarial example: {adv_text}")

Adversarial example: This mosvie was agreat!
Adversarial example: The acting was terrile
Adversarial example: I lnoved this film
Adversarial example: It was a waset of time
Adversarial example: Teh polt was predictabel
Adversarial example: This is a musZ-watch omvie
Adversarial example: I wuold not recymmend this moviT
Adversarial example: The cinematography was stunniPng
Adversarial example: The script was weal
Adversarial example: Teh enging was dsappointing
Adversarial example: This is the bets move veer!
Adversarial example: I rfegret watching this film
Adversarial example: Teh charatcers were well-developed
Adversarial example: It was too slow-pcaed
Adversarial example: I was on the edge of my seat the whole time
Adversarial example: The special effects were amazin
Adversarial example: I was boed throughout the entire movie
Adversarial example: The soudntrack was beutiful
Adversarial example: The dialogue was chesey
Adversarial example: I owuld awtch this moKvie again and again
Ad

In [ ]:
import csv
csv_file_path = '/content/drive/MyDrive/Model/DeepWordBugGao2018_adv_examples.csv'
# Write the adversarial examples to the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['review', 'sentiment']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for example in adv_texts:
        writer.writerow({'review': example[0], 'sentiment': example[1]})

In [ ]:
# Tokenize the adversarial texts
adv_inputs = tokenizer(list(adv_texts), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Evaluate accuracy using the adversarial examples
adv_preds = model_wrapper.model(adv_inputs).logits.numpy().argmax(axis=1)
adv_acc = (adv_preds == labels).mean()

print(f"Adversarial accuracy: {adv_acc:.4f}")

Adversarial accuracy: 0.3300


In [ ]:
# Create DataFrame with adv_texts and labels
df_adv1 = pd.DataFrame({'review': adv_texts, 'sentiment': labels})

In [ ]:
df_adv1['review'] = df_adv1['review'].apply(preprocess_text)
adv_df1 = df_adv1[df_adv1['review'].str.split().str.len() < 100]

In [ ]:
# Create a new dataframe containing both original and adversarial examples
df_adv1 = pd.concat([df1, adv_df1], ignore_index=True)
df_adv1.to_csv('/content/drive/MyDrive/Model/DeepWordBugGao2018_added_adv_examples.csv', index=False)

In [ ]:
df_adv1 = pd.read_csv('/content/drive/MyDrive/Model/DeepWordBugGao2018_adv_examples.csv')

In [ ]:
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df_adv1['review']), padding=True, truncation=True, max_length=100)
X_train1 = train_encodings['input_ids']
y_train1 = np.array(list(df_adv1['sentiment']))

In [ ]:
# Train the model
train_dataset1 = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train1, "attention_mask": np.ones_like(X_train1)},
    y_train1
))
train_dataset1 = train_dataset1.shuffle(len(X_train1)).batch(32)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
history = model.fit(
    train_dataset1,
    validation_data=val_dataset,
    epochs=2,
    steps_per_epoch=len(X_train1) // 32
)

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Model/Model-Trained-DeepWordBugGao2018')

In [ ]:
attack = InputReductionFeng2018.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.


In [ ]:
adv_texts = []
for text, label in zip(texts, labels):
    results_iterable = attack.attack(text, ground_truth_output=int(label))
    if hasattr(results_iterable, "__iter__"):
        for result in results_iterable:
            adv_text = results_iterable.perturbed_text()
            adv_texts.append(adv_text)
            print(f"Adversarial example: {adv_text}")
    else:
        adv_text = results_iterable.perturbed_text()
        adv_texts.append(adv_text)
        print(f"Adversarial example: {adv_text}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Adversarial example: great!
Adversarial example: acting terrible
Adversarial example: loved 
Adversarial example: was waste of 
Adversarial example: was predictable
Adversarial example: must-watch movie
Adversarial example: would this 
Adversarial example: cinematography 
Adversarial example: was weak
Adversarial example: was disappointing
Adversarial example: the movie!
Adversarial example: regret watching film
Adversarial example: characters 
Adversarial example: was slow-paced
Adversarial example: I was on the edge of my seat the whole time
Adversarial example: The special were 
Adversarial example: was throughout 
Adversarial example: soundtrack 
Adversarial example: dialogue was cheesy
Adversarial example: I would watch this movie again and again
Adversarial example: story 
Adversarial example: was a 
Adversarial example: The was superb
Adversarial example: couldn't 
Adversarial example: cast job
Adversarial example: was a total of 
Adversarial example: thought-provoking
Adversari

In [ ]:
# Tokenize the adversarial texts
adv_inputs = tokenizer(list(adv_texts), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Evaluate accuracy using the adversarial examples
adv_preds = model_wrapper.model(adv_inputs).logits.numpy().argmax(axis=1)
adv_acc = (adv_preds == labels).mean()

print(f"Adversarial accuracy: {adv_acc:.4f}")

Adversarial accuracy: 0.8637


In [ ]:
df_adv2 = pd.DataFrame({'review': adv_texts, 'sentiment': labels})

In [ ]:
df_adv2['review'] = df_adv2['review'].apply(preprocess_text)
adv_df2 = df_adv2[df_adv2['review'].str.split().str.len() < 100]

In [ ]:
# Create a new dataframe containing both original and adversarial examples
df_adv2 = pd.concat([df1, adv_df2], ignore_index=True)
df_adv2.to_csv('/content/drive/MyDrive/Model/InputReductionFeng2018_added_adv_examples.csv', index=False)

In [ ]:
df_adv2 = pd.read_csv('/content/drive/MyDrive/Model/InputReductionFeng2018_added_adv_examples.csv')

In [ ]:
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df_adv2['review']), padding=True, truncation=True, max_length=100)
X_train2 = train_encodings['input_ids']
y_train2 = np.array(list(df_adv2['sentiment']))

In [ ]:
# Train the model
train_dataset2 = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train2, "attention_mask": np.ones_like(X_train2)},
    y_train2
))
train_dataset2 = train_dataset2.shuffle(len(X_train2)).batch(32)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
history = model.fit(
    train_dataset2,
    validation_data=val_dataset,
    epochs=2,
    steps_per_epoch=len(X_train2) // 32
)

Epoch 1/2
222/222 [==============================] - 208s 701ms/step - loss: 0.3305 - accuracy: 0.8519 - val_loss: 0.0827 - val_accuracy: 0.9869
Epoch 2/2
  1/222 [..............................] - ETA: 1:28 - loss: 0.3763 - accuracy: 0.8667

222/222 [==============================] - 2s 5ms/step - loss: 0.3763 - accuracy: 0.8667 - val_loss: 0.0821 - val_accuracy: 0.9869
